https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver

In [6]:
!pip install qiskit_nature qiskit qiskit_ibm_runtime pylatexenc pyscf qiskit_algorithms

import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

In [8]:
# runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)

### calculating qubit hamiltonian (use with more computational resources)

In [9]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

In [10]:
driver = PySCFDriver(
    atom="O 0 0 0; H 0.757 0.586 0; H -0.757 0.586 0",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
print(problem)

hamiltonian = problem.hamiltonian.second_q_op()

mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(hamiltonian)

In [17]:
print(qubit_hamiltonian)
print(type(qubit_hamiltonian))
hamiltonian=qubit_hamiltonian
#hamiltonian = SparsePauliOp.from_op(qubit_hamiltonian)

# hamiltonian = SparsePauliOp.from_list(
#     [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
# )

SparsePauliOp(['IIIIIIIIIIIIII', 'IIIIIIIIIIIIIZ', 'IIIIIIIIIIIIYY', 'IIIIIIIIIIIIXX', 'IIIIIIIIIIYZZY', 'IIIIIIIIIIXZZX', 'IIIIIIIIYZZZZY', 'IIIIIIIIXZZZZX', 'IIIIIIIIIIIIZI', 'IIIIIIIIIIYZYI', 'IIIIIIIIIIXZXI', 'IIIIIIIIYZZZYI', 'IIIIIIIIXZZZXI', 'IIIIIIIIIIIZII', 'IIIIIIIYZZZYII', 'IIIIIIIXZZZXII', 'IIIIIIIIIIZIII', 'IIIIIIIIYZYIII', 'IIIIIIIIXZXIII', 'IIIIIIIIIZIIII', 'IIIIIIIIZIIIII', 'IIIIIIIZIIIIII', 'IIIIIIZIIIIIII', 'IIIIIYYIIIIIII', 'IIIIIXXIIIIIII', 'IIIYZZYIIIIIII', 'IIIXZZXIIIIIII', 'IYZZZZYIIIIIII', 'IXZZZZXIIIIIII', 'IIIIIZIIIIIIII', 'IIIYZYIIIIIIII', 'IIIXZXIIIIIIII', 'IYZZZYIIIIIIII', 'IXZZZXIIIIIIII', 'IIIIZIIIIIIIII', 'YZZZYIIIIIIIII', 'XZZZXIIIIIIIII', 'IIIZIIIIIIIIII', 'IYZYIIIIIIIIII', 'IXZXIIIIIIIIII', 'IIZIIIIIIIIIII', 'IZIIIIIIIIIIII', 'ZIIIIIIIIIIIII', 'IIIIIIIIIIIIZZ', 'IIIIIIIIIIYZYZ', 'IIIIIIIIIIXZXZ', 'IIIIIIIIYZZZYZ', 'IIIIIIIIXZZZXZ', 'IIIIIIIIIIIZIZ', 'IIIIIIIYZZZYIZ', 'IIIIIIIXZZZXIZ', 'IIIIIIIIIIZIIZ', 'IIIIIIIIYZYIIZ', 'IIIIIIIIXZXIIZ', 'IIIIIIIIIZII

In [18]:
ansatz = EfficientSU2(hamiltonian.num_qubits)

In [20]:
num_params = ansatz.num_parameters
num_params

112

In [21]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

ansatz_isa = pm.run(ansatz)

### ISA observable


In [22]:
hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)

In [26]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

def cost_func(params, ansatz, hamiltonian, estimator, max_iters=10):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    if cost_history_dict["iters"] >= max_iters:
        print(f"max iter.")
        return cost_history_dict["cost_history"][-1]
    
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    #print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [27]:
x0 = 2 * np.pi * np.random.random(num_params)

In [28]:
x0

array([2.33339579, 1.38980993, 5.56606121, 0.94012453, 0.37406898,
       6.00693736, 4.5350989 , 2.14397538, 2.02986092, 5.57803543,
       3.27227333, 5.35234695, 3.81049911, 2.20289283, 2.4726176 ,
       4.82408018, 0.29966231, 4.76837719, 1.54194883, 4.46557166,
       5.13435462, 5.78473155, 0.34762651, 4.88085531, 2.1537556 ,
       2.62352752, 2.05932231, 4.50506189, 3.40911728, 3.90042599,
       4.44494074, 5.06072015, 1.19268224, 3.74638083, 2.00576935,
       2.86502683, 3.14502472, 0.82154107, 4.22660574, 3.69647036,
       5.07454422, 5.51188304, 2.5401033 , 2.28230677, 4.7673244 ,
       0.39941322, 0.92635305, 0.01963485, 2.6807701 , 4.49826522,
       2.29886146, 2.7218021 , 2.55957054, 4.876838  , 1.0699734 ,
       4.85731907, 2.78973769, 4.40235023, 0.14102515, 3.5552049 ,
       3.04259707, 1.29666858, 2.0033172 , 1.47165066, 4.29088445,
       5.62409579, 5.31281663, 1.67089564, 4.44378061, 5.22879831,
       5.55454174, 1.67655238, 0.2007064 , 0.86422045, 3.21898

In [ ]:
with Session(backend=backend) as session:
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000

    res = minimize(
        cost_func,
        x0,
        args=(ansatz_isa, hamiltonian_isa, estimator),
        method="cobyla",
    )

In [30]:
res

10 iters optimization: 73.62959421
